<a href="https://colab.research.google.com/github/vedangi-jawade/RFM-Analysis/blob/main/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data.csv')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
12457,537381,22632,HAND WARMER RED RETROSPOT,24.0,12/6/2010 13:13,2.10,14667.0,United Kingdom
12458,537381,22633,HAND WARMER UNION JACK,20.0,12/6/2010 13:13,2.10,14667.0,United Kingdom
12459,537381,22867,HAND WARMER BIRD DESIGN,24.0,12/6/2010 13:13,2.10,14667.0,United Kingdom
12460,537381,22601,CHRISTMAS RETROSPOT ANGEL WOOD,17.0,12/6/2010 13:13,0.85,14667.0,United Kingdom


In [5]:
df = df.dropna()
df = df.drop_duplicates()

# converting InvoiceDate column to pandas datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# sorting records by InvoiceDate in ascending order
df = df.sort_values(by='InvoiceDate')

# removing trailing '.0' from CustomerID
df['CustomerID'] = df['CustomerID'].apply(lambda x: str(x)[:-2])

# add column Amount = quantity * unit_price
df['Amount'] = round(df['Quantity']*df['UnitPrice'],2)

In [6]:
# taking only necessary columns and aggregating by 'CustomerID'
df = df[['CustomerID','InvoiceDate','Amount']]
rfm_df = df.groupby('CustomerID').agg(list).reset_index()
rfm_df.head(5)

,CustomerID,InvoiceDate,Amount
0,12395,"[2010-12-03 16:35:00, 2010-12-03 16:35:00, 201...","[19.5, 36.0, 13.2, 8.5, 26.4, 19.8, 50.4, 50.4..."
1,12427,"[2010-12-03 10:44:00, 2010-12-03 10:44:00, 201...","[15.3, 19.8, 54.0, 19.8, 15.3, 19.8, 61.2, 17...."
2,12431,"[2010-12-01 10:03:00, 2010-12-01 10:03:00, 201...","[20.4, 39.6, 17.0, 17.0, 15.0, 17.0, 15.0, 35...."
3,12433,"[2010-12-01 13:24:00, 2010-12-01 13:24:00, 201...","[25.2, 13.2, 13.2, 13.2, 13.2, 90.0, 30.0, 10...."
4,12471,[2010-12-02 10:37:00],[-17.0]


In [7]:
# find last dates for every customer
rfm_df['last_dates'] = rfm_df['InvoiceDate'].apply(lambda x: x[-1])

# find latest date of purchase
latest_date = df['InvoiceDate'].max()

# find recency by subtracting last_dates from maximum date
rfm_df['recency'] = rfm_df['last_dates'].apply(lambda x: (latest_date- x).days)

In [8]:
# find frequency by counting no. of transactions per customer
rfm_df['frequency'] = rfm_df['Amount'].apply(len)

# find monetary value by summing up transaction amounts per customer
rfm_df['monetary_value'] = rfm_df['Amount'].apply(sum)

# take only necessary columns
rfm_df=rfm_df[['CustomerID','recency','frequency','monetary_value']]

In [9]:
# binning recency col
rfm_df['r'] = pd.qcut(rfm_df['recency'].rank(method='first'),5,labels=[5,4,3,2,1]).tolist()

# binning frequency col
rfm_df['f'] = pd.qcut(rfm_df['frequency'].rank(method='first'),5,labels=[1,2,3,4,5]).tolist()

# binning monetary_value col
rfm_df['m'] = pd.qcut(rfm_df['monetary_value'].rank(method='first'),5,labels=[1,2,3,4,5]).tolist()

# concatenating r,f,m scores together
rfm_df['rfm_score'] = rfm_df['r'].apply(str) + rfm_df['f'].apply(str) + rfm_df['m'].apply(str)

In [12]:
def find_segments(df, customer_id):
    classes = []
    for row in df.iterrows():
        if (row[1]['r'] in [4,5]) & (row[1]['f'] in [4,5]) & (row[1]['m'] in [4,5]):
            classes.append({row[1][customer_id]:'Champions'})
        elif (row[1]['r'] in [4,5]) & (row[1]['f'] in [1,2]) & (row[1]['m'] in [3,4,5]):
            classes.append({row[1][customer_id]:'Promising'})
        elif (row[1]['r'] in [3,4,5]) & (row[1]['f'] in [3,4,5]) & (row[1]['m'] in [3,4,5]):
            classes.append({row[1][customer_id]:'Loyal Accounts'})
        elif (row[1]['r'] in [3,4,5]) & (row[1]['f'] in [2,3]) & (row[1]['m'] in [2,3,4]):
            classes.append({row[1][customer_id]:'Potential Loyalist'})
        elif (row[1]['r'] in [5]) & (row[1]['f'] in [1]) & (row[1]['m'] in [1,2,3,4,5]):
            classes.append({row[1][customer_id]:'New Active Accounts'})
        elif (row[1]['r'] in [3,4,5]) & (row[1]['f'] in [1,2,3,4,5]) & (row[1]['m'] in [1,2]):
            classes.append({row[1][customer_id]:'Low Spenders'})
        elif (row[1]['r'] in [2,3]) & (row[1]['f'] in [1,2]) & (row[1]['m'] in [4,5]):
            classes.append({row[1][customer_id]:'Need Attention'})
        elif (row[1]['r'] in [2,3]) & (row[1]['f'] in [1,2]) & (row[1]['m'] in [1,2,3]):
            classes.append({row[1][customer_id]:"About to Sleep"})
        elif (row[1]['r'] in [1,2]) & (row[1]['f'] in [1,2,3,4,5]) & (row[1]['m'] in [3,4,5]):
            classes.append({row[1][customer_id]:'At Risk'})
        elif (row[1]['r'] in [1,2]) & (row[1]['f'] in [1,2,3,4,5]) & (row[1]['m'] in [1,2]):
            classes.append({row[1][customer_id]:"Lost"})
        else:
            classes.append({0:[row[1]['r'],row[1]['f'],row[1]['m']]})
    accs = [list(i.keys())[0] for i in classes]
    segments = [list(i.values())[0] for i in classes]
    df['segment'] = df[customer_id].map(dict(zip(accs,segments)))
    return df

rfm_df = find_segments(rfm_df, 'CustomerID')
rfm_df.head(10)

,CustomerID,recency,frequency,monetary_value,r,f,m,rfm_score,segment
0,12395,2,12,346.10,4,2,4,424,Promising
1,12427,3,10,303.50,3,2,3,323,Potential Loyalist
2,12431,5,14,358.25,1,3,4,134,At Risk
3,12433,4,73,1919.14,2,5,5,255,At Risk
4,12471,4,1,-17.00,2,1,1,211,About to Sleep
5,12472,0,84,1509.00,5,5,5,555,Champions
6,12474,0,1,-34.00,5,1,1,511,New Active Accounts
7,12557,3,5,620.00,3,1,5,315,Need Attention
8,12567,1,63,1526.92,4,5,5,455,Champions
9,12583,5,20,855.86,1,4,5,145,At Risk
